In [59]:
import os
from langchain.document_loaders import PyMuPDFLoader
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

PDF_NAME = '../serverless-core.pdf'

docs = PyMuPDFLoader(PDF_NAME).load()
print (f'There are {len(docs)} document(s) in {PDF_NAME}.')
print (f'There are {len(docs[0].page_content)} characters in the first page of your document.')


There are 91 document(s) in ../serverless-core.pdf.
There are 112 characters in the first page of your document.


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
# import chromadb


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(split_docs, embeddings, collection_name="serverless_guide")


In [ ]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

llm = OpenAI(temperature=0)
chain = load_qa_chain(llm, chain_type="stuff")

基于提问，进行相似性查询

In [ ]:
query = "What is the use case of AWS Serverless?"
similar_docs = vectorstore.similarity_search(query, 3, include_metadata=True)

基于相关文档，利用QA链完成回答

In [ ]:
chain.run(input_documents=similar_docs, question=query)